# Encounter Rate Experiment Notebook with Function Explanation

In [1]:
using LinearAlgebra, Statistics, DataFrames, CSV, Tables, Dates
using PlanktonIndividuals, Plots
import JLD2

## Optimal Movement Algorithm
A function to calculate the optimal movement vector for prey to avoid multiple predators in 3D. Predator movement does not need a special algorithm, since predators only move towards the nearest prey or at a random vector. Therefore, predator movement is handled in an if-else-then statement later in the code.

### Function Goals
1) Calculate the distance and direction vector for each predator

#
    direction_vector = transpose(prey_position) .- predator_positions
    distances = sum(direction_vector.^2, dims=2)
    closest_predator = argmin(distances)
#

2) Normalize the direction vectors

#
    direction_vector = direction_vector[closest_predator, :]
    direction_vector /= sqrt(sum(direction_vector.^2))
#

3) Calculate the fraction of the full possible movement distance that the prey needs to move to reach the optimal distance

#
    average_distance = mean(sqrt.(sum((transpose(prey_position) .- predator_positions).^2, dims=2)))
    move_frac = min(1,1.0 - 0.1 * average_distance)
    move_distance = move_frac * min(avoidance_distance, sqrt(sum(direction_vector.^2)))
#


4) Identify the prey's optimal location

#
    optimal_movement = prey_position .+ direction_vector * move_distance
#

### Necessary Variables
- prey_position: The location of the prey that is to be moved
- predator_positions: A dataframe with the locations of all predators within visual range
- avoidance_distance: The maximum distance that an individual prey can move.


In [ ]:
function optimal_movement(prey_position, predator_positions, avoidance_distance)
    # Calculate direction away from all predators

    direction_vector = transpose(prey_position) .- predator_positions
    distances = sum(direction_vector.^2, dims=2)
    closest_predator = argmin(distances)
  
    direction_vector = direction_vector[closest_predator, :]
    direction_vector /= sqrt(sum(direction_vector.^2))  # Normalize
    
    
    # Example: Move fraction decreases as the average distance to predators increases
    average_distance = mean(sqrt.(sum((transpose(prey_position) .- predator_positions).^2, dims=2)))

    move_frac = min(1,1.0 - 0.1 * average_distance)
  
    # Calculate distance to move (avoidance distance)
    move_distance = move_frac * min(avoidance_distance, sqrt(sum(direction_vector.^2)))
  
    # Update prey's position
    optimal_movement = prey_position .+ direction_vector * move_distance
  
    return optimal_movement
end

## Plot model function
This function drives most of what is actually happening in the model. The code also constains plotting capabilities, which can be commented out by the user.

### Function Goals
1) Set up results and find all preys
#
    prey_num = []
    encounter_tresh_sq = encounter_tresh^2
    ip_ac = model.individuals.phytos.sp1.data.ac .== 1.0
    p_pos = transpose(hcat(Array(model.individuals.phytos.sp1.data.x[ip_ac]), Array(model.individuals.phytos.sp1.data.y[ip_ac]), Array(model.individuals.phytos.sp1.data.z[ip_ac])))
    iz_ac = model.individuals.phytos.sp2.data.ac .== 1.0
    z_pos = transpose(hcat(Array(model.individuals.phytos.sp2.data.x[iz_ac]), Array(model.individuals.phytos.sp2.data.y[iz_ac]), Array(model.individuals.phytos.sp2.data.z[iz_ac])))
    ip_ac_int = findall(ip_ac)
    iz_ac_int = findall(iz_ac)
#
2) Calculate the distance between each predator and prey
#
    dist_3d = zeros(Float64,length(ip_ac_int),length(iz_ac_int))
    for i in 1:size(dist_3d,1)
        for j in 1:size(dist_3d,2)
                dist_3d[i,j] = sqrt((p_pos[1,i]-z_pos[1,j])^2+(p_pos[2,i]-z_pos[2,j])^2+(p_pos[3,i]-z_pos[3,j])^2)
        end
    end
#
3) Move the preys to the optimal location, but do not calculate any encounters yet. If a prey cannot perceive a predator, it will move at a random vector. Otherwise, it will call the optimal_movement function
#
    if velo2 > 0.0
            vec = Array{Float64}(undef, 3)
            max_movement_len = ΔT * velo2
            max_movement_len_sq = max_movement_len
            for (ip, iz_model) in enumerate(ip_ac_int)
                iz = argmin(dist_3d[ip, :]) #Find closest predator
                prey_visual_range = ((3 / (4 * π)) * prey_vis)^(1/3)
                if dist_3d[ip, iz] > prey_visual_range
                    for idim in 1:3
                        vec[idim] = 2* rand() -1
                    end
                    p_pos[:,ip] = p_pos[:,ip] .+ vec .* max_movement_len
                else
                    prey_position = [model.individuals.phytos.sp1.data.x[ip],model.individuals.phytos.sp1.data.y[ip],model.individuals.phytos.sp1.data.z[ip]]
                    ind_dists = dist_3d[ip,:]
                    sub_dist = ind_dists[ind_dists .< prey_visual_range]
                    pred_positions = Matrix{Float64}(undef,size(sub_dist,1),3)
                    sort_vector =sort(dist_3d[ip,:])
                    if size(sub_dist,1) > 0
                        for i in 1:size(sub_dist,1)
                            pred_dist = sort_vector[i] #Next closest prey
                            index = findall(x -> x == pred_dist,ind_dists)
                            x = model.individuals.phytos.sp2.data.x[index][1]
                            y = model.individuals.phytos.sp2.data.y[index][1]
                            z = model.individuals.phytos.sp2.data.z[index][1]
                            pred_positions[i,:] .= (x,y,z)
                        end
                        max_distance = max_movement_len_sq #Can complicate this to correspond to a flexible time frame 
                        optimal_location = optimal_movement(prey_position,pred_positions,max_distance)
                        p_pos[:, ip] = optimal_location
                    end
                end
                model.individuals.phytos.sp1.data.x[iz_model] = p_pos[1, ip]
                model.individuals.phytos.sp1.data.y[iz_model] = p_pos[2, ip]
                model.individuals.phytos.sp1.data.z[iz_model] = p_pos[3, ip]
            end
        end
#
4) Find all individuals and recalculate distances
#
    ip_ac = model.individuals.phytos.sp1.data.ac .== 1.0
    p_pos = transpose(hcat(Array(model.individuals.phytos.sp1.data.x[ip_ac]), Array(model.individuals.phytos.sp1.data.y[ip_ac]), Array(model.individuals.phytos.sp1.data.z[ip_ac])))
    iz_ac = model.individuals.phytos.sp2.data.ac .== 1.0
    z_pos = transpose(hcat(Array(model.individuals.phytos.sp2.data.x[iz_ac]), Array(model.individuals.phytos.sp2.data.y[iz_ac]), Array(model.individuals.phytos.sp2.data.z[iz_ac])))
    ip_ac_int = findall(ip_ac)
    iz_ac_int = findall(iz_ac)
    for i in 1:size(dist_3d,1)
        for j in 1:size(dist_3d,2)
            dist_3d[i,j] = sqrt((p_pos[1,i]-z_pos[1,j])^2+(p_pos[2,i]-z_pos[2,j])^2+(p_pos[3,i]-z_pos[3,j])^2)
        end
    end
#
5) Move predators towards the nearest prey or to a random vector. If the predator has recently eaten, within the handing time, most of this function will be skipped. This segment will also calculate the number of preys within each predator's visual field at iteration number 300 (currently the last iteration in the simulation).
# 
    if velo1 > 0.0
        vec = Array{Float64}(undef, 3)
        max_movement_len = ΔT * velo1
        max_movement_len_sq = max_movement_len
        for (iz, iz_model) in enumerate(iz_ac_int)
            if z_pause[iz_model] > 0.0
                z_pause[iz_model] = max(0.0, z_pause[iz_model] - ΔT)
                continue
            end
            ip = argmin(dist_3d[:, iz])
            pred_visual_range = ((3 / (4 * π)) * pred_vis)^(1/3)
            if iteration == 300
                densities[iz,isubexp] = length(dist_3d[:,iz][dist_3d[:,iz] .< pred_visual_range])
            end
            count = 1
            if dist_3d[ip, iz] ≤ max_movement_len_sq
                z_pos[:, iz] = p_pos[:, ip]
            elseif dist_3d[ip,iz] ≤ pred_visual_range
                vec .= p_pos[:, ip] .- z_pos[:, iz]
                normalized_movement_vector = vec / dist_3d[ip,iz]
                z_pos[:, iz] = z_pos[:,iz] .+ normalized_movement_vector .* max_movement_len
            else
                for idim in 1:3
                    vec[idim] = 2* rand() -1
                end
                z_pos[:, iz] = z_pos[:,iz] .+ vec .* max_movement_len
            end
            model.individuals.phytos.sp2.data.x[iz_model] = z_pos[1, iz]
            model.individuals.phytos.sp2.data.y[iz_model] = z_pos[2, iz]
            model.individuals.phytos.sp2.data.z[iz_model] = z_pos[3, iz]
        end
    else
        ind_z = z_pause .> 0
        z_pause[ind_z] = max.(0.0, z_pause[ind_z] .- ΔT)
    end
#
6) Calculate encounters based on distance and the encounter threshold and create the new plot object with updated locations. Sets the handling time for predators that just consumed prey.
#
    for i in 1:size(dist_3d,1)
        for j in 1:size(dist_3d,2)
            dist_3d[i,j] = sqrt((p_pos[1,i]-z_pos[1,j])^2+(p_pos[2,i]-z_pos[2,j])^2+(p_pos[3,i]-z_pos[3,j])^2)
        end
    end
    plt = Plots.scatter(p_pos[1, :], p_pos[2, :], p_pos[3,:], ms=4, color="#228833", markerstrokewidth=0, size=(900, 900), legend=:none)
    Plots.scatter!(z_pos[1, :], z_pos[2, :], z_pos[3,:], ms=5, color="#CCBB44", markerstrokewidth=0, legend=:none)
    encounters = findall(dist_3d .< encounter_tresh_sq)
    iz_vec = [ci.I[2] for ci in encounters]
    Plots.scatter!(z_pos[1, iz_vec], z_pos[2, iz_vec], z_pos[3,iz_vec], ms=5, color = "red", legend=:none)
    num_encounters = 0
    for ci in encounters
        if z_pause[iz_ac_int[ci.I[2]]] > 0.0
            continue
        end
        Plots.scatter!([p_pos[1, ci.I[1]], z_pos[1, ci.I[2]]], [p_pos[2, ci.I[1]], z_pos[2, ci.I[2]]],[p_pos[3, ci.I[1]], z_pos[3, ci.I[2]]], ms=5, color="#EE6677", markerstrokewidth=0, legend=:none)
        Plots.scatter!([z_pos[1, ci.I[2]]], [z_pos[2, ci.I[2]]], [z_pos[3, ci.I[2]]], ms=5, color="#EE6677", markerstrokewidth=0, legend=:none)
        if deactivate
            #println("deactivating phytoplankton $(ip_ac_int[ci.I[1]])")
            model.individuals.phytos.sp1.data.ac[ip_ac_int[ci.I[1]]] = 0.0
        else
            model.individuals.phytos.sp1.data.x[ip_ac_int[ci.I[1]]] = rand()
            model.individuals.phytos.sp1.data.y[ip_ac_int[ci.I[1]]] = rand()
            model.individuals.phytos.sp1.data.z[ip_ac_int[ci.I[1]]] = rand()
        end
        z_pause[iz_ac_int[ci.I[2]]] = z_pause_ΔT
        num_encounters += 1
    end
    push!(encounter_diags["num_encounters"], num_encounters)
    Plots.xlims!(0, 1)
    Plots.ylims!(0, 1)
    Plots.zlims!(0, 1)
#

### Necessary Variables.
- model: The model object that contains state varables. It's structure comes from the PlanktonIndividuals.jl package
- z_pause: A vector that controls the handling time of each predator since the handling time may be greater than the temporal resolution of the model.
- ΔT: The temporal resolution of the model
- deactivate: A boolean object that allows the user to turn specific plankton objects off once they are consumed. In this model run, deactivate is false and prey abundance remains constant
- encounter_diags: A dictionary that stores information about encounters that have occurred throughout the simulation
- densities: A matrix of predator-specific prey densities that follows through each simulation.
- isubexp: A counter of the simulation number that is useful to control the column in the densities matrix
- iteration: A counter for the iteration of that particular simulation
- z_pause_ΔT: The handling time after a predator successfully consumes prey
- velo1: The predator swimming velocity in m/s. The model domain will *always* be a non-dimensional 0-1, so velocity values should be scaled accordingly.
- velo2: The prey swimming velocity in m/s. The model domain will *always* be a non-dimensional 0-1, so velocity values should be scaled accordingly.
- encounter_thresh: The distance required for an encounter to occur.
- prey_vis: The visual acuity of the prey provided as a proportion of the total model volume
- pred_vis: The visual acuity of the predator provided as a proportion of the total model volume

In [ ]:
function plot_model(model::PlanktonModel, z_pause::Vector{Float64}, ΔT::Float64, deactivate::Bool, encounter_diags::Dict{String,Any},densities::Matrix{Int},isubexp::Int64,iteration::Int64; z_pause_ΔT=0.0::Float64, velo1=1e-4::Float64, velo2 = 1e-5::Float64, encounter_tresh=0.001::Float64,prey_vis=1::Float64, pred_vis = 1::Float64)

    # Blank count for prey numbers
    prey_num = []

    # square here as sqrt is not taken in computation of distance
    encounter_tresh_sq = encounter_tresh^2

    # ac stands for active
    ip_ac = model.individuals.phytos.sp1.data.ac .== 1.0
    p_pos = transpose(hcat(Array(model.individuals.phytos.sp1.data.x[ip_ac]), Array(model.individuals.phytos.sp1.data.y[ip_ac]), Array(model.individuals.phytos.sp1.data.z[ip_ac])))
    
    iz_ac = model.individuals.phytos.sp2.data.ac .== 1.0
    z_pos = transpose(hcat(Array(model.individuals.phytos.sp2.data.x[iz_ac]), Array(model.individuals.phytos.sp2.data.y[iz_ac]), Array(model.individuals.phytos.sp2.data.z[iz_ac])))
    
    # convert logical to integer array
    ip_ac_int = findall(ip_ac)
    iz_ac_int = findall(iz_ac)
    
    
    # compute distance
    dist_3d = zeros(Float64,length(ip_ac_int),length(iz_ac_int))

    for i in 1:size(dist_3d,1)
        for j in 1:size(dist_3d,2)
                dist_3d[i,j] = sqrt((p_pos[1,i]-z_pos[1,j])^2+(p_pos[2,i]-z_pos[2,j])^2+(p_pos[3,i]-z_pos[3,j])^2)
        end
    end

    ## Prey movement (Does not calculate encounters.)
    if velo2 > 0.0
        vec = Array{Float64}(undef, 3)
        max_movement_len = ΔT * velo2
        # square here as sqrt was not taken in computation of distance
        max_movement_len_sq = max_movement_len

        # find closest and build vector
        for (ip, iz_model) in enumerate(ip_ac_int)
            iz = argmin(dist_3d[ip, :]) #Find closest predator

            #@assert z_pos[1, iz] == model.individuals.phytos.sp2.data.x[iz_model]
            #@assert z_pos[2, iz] == model.individuals.phytos.sp2.data.y[iz_model]
            #@assert z_pos[3, iz] == model.individuals.phytos.sp2.data.z[iz_model]

            prey_visual_range = ((3 / (4 * π)) * prey_vis)^(1/3) #Calculates the radial distance the prey can visualize based on the proportion provided.
            
            #Placeholder
            # check if predators are within range. Need to change so predator finds "optimal location".

            if dist_3d[ip, iz] > prey_visual_range #Animal moves to random vector because closest animal is outside of visual range
                for idim in 1:3
                    vec[idim] = 2* rand() -1 #Animal moves to random vector between -1 and 1

                    #if dist_periodic[idim, ip, iz] #Remove boundary effects?
                    #    println(dist_periodic[idim,ip,iz])
                    #    throw(ErrorException("stop"))
                    #    if vec[idim] > 0.0
                    #        vec[idim] = vec[idim] - 1.0
                    #    else
                    #        vec[idim] = vec[idim] + 1.0
                    #    end
                    #end
                end

                #vec /= sqrt(sum(vec.^2))

                p_pos[:,ip] = p_pos[:,ip] .+ vec .* max_movement_len

            else #Animal makes decision to move to "optimal" location.
                #Prey position
                prey_position = [model.individuals.phytos.sp1.data.x[ip],model.individuals.phytos.sp1.data.y[ip],model.individuals.phytos.sp1.data.z[ip]]


                #Find all preds within visual range
                ind_dists = dist_3d[ip,:]
                sub_dist = ind_dists[ind_dists .< prey_visual_range]

                pred_positions = Matrix{Float64}(undef,size(sub_dist,1),3)
                sort_vector =sort(dist_3d[ip,:])

                if size(sub_dist,1) > 0
                    for i in 1:size(sub_dist,1)

                        pred_dist = sort_vector[i] #Next closest prey
                        index = findall(x -> x == pred_dist,ind_dists)

                        #Gather x,y,z coordinates of next closest predator
                        x = model.individuals.phytos.sp2.data.x[index][1]
                        y = model.individuals.phytos.sp2.data.y[index][1]
                        z = model.individuals.phytos.sp2.data.z[index][1]

                        pred_positions[i,:] .= (x,y,z)
                    end

                    max_distance = max_movement_len_sq #Can complicate this to correspond to a flexible time frame 

                    optimal_location = optimal_movement(prey_position,pred_positions,max_distance)

                    p_pos[:, ip] = optimal_location
                end
            end

            #Update location
            model.individuals.phytos.sp1.data.x[iz_model] = p_pos[1, ip]
            model.individuals.phytos.sp1.data.y[iz_model] = p_pos[2, ip]
            model.individuals.phytos.sp1.data.z[iz_model] = p_pos[3, ip]

        end
    end

    ## Predator movement (Calculates encounters; i.e., predators catch up to prey)
    
    # ac stands for active
    ip_ac = model.individuals.phytos.sp1.data.ac .== 1.0
    p_pos = transpose(hcat(Array(model.individuals.phytos.sp1.data.x[ip_ac]), Array(model.individuals.phytos.sp1.data.y[ip_ac]), Array(model.individuals.phytos.sp1.data.z[ip_ac])))

    iz_ac = model.individuals.phytos.sp2.data.ac .== 1.0
    z_pos = transpose(hcat(Array(model.individuals.phytos.sp2.data.x[iz_ac]), Array(model.individuals.phytos.sp2.data.y[iz_ac]), Array(model.individuals.phytos.sp2.data.z[iz_ac])))


    # convert logical to integer array
    ip_ac_int = findall(ip_ac)
    iz_ac_int = findall(iz_ac)

    # compute distance
    for i in 1:size(dist_3d,1)
        for j in 1:size(dist_3d,2)
            dist_3d[i,j] = sqrt((p_pos[1,i]-z_pos[1,j])^2+(p_pos[2,i]-z_pos[2,j])^2+(p_pos[3,i]-z_pos[3,j])^2)
        end
    end

    if velo1 > 0.0
        vec = Array{Float64}(undef, 3)
        max_movement_len = ΔT * velo1
        # square here as sqrt was not taken in computation of distance
        max_movement_len_sq = max_movement_len

        # find closest and build vector
        for (iz, iz_model) in enumerate(iz_ac_int)
            if z_pause[iz_model] > 0.0
                z_pause[iz_model] = max(0.0, z_pause[iz_model] - ΔT)
                continue
            end
            ip = argmin(dist_3d[:, iz])

            #@assert z_pos[1, iz] == model.individuals.phytos.sp2.data.x[iz_model]
            #@assert z_pos[2, iz] == model.individuals.phytos.sp2.data.y[iz_model]
            #@assert z_pos[3, iz] == model.individuals.phytos.sp2.data.z[iz_model]

            pred_visual_range = ((3 / (4 * π)) * pred_vis)^(1/3) #Calculates the radial distance the prey can visualize based on the proportion provided.

            if iteration == 300  #Want the final result of the prey densities to show the effect
                densities[iz,isubexp] = length(dist_3d[:,iz][dist_3d[:,iz] .< pred_visual_range])
            end
            count = 1
            # check if closest phytoplankton can be reached in this timestep
            if dist_3d[ip, iz] ≤ max_movement_len_sq
                #println("moving zooplankton $(iz_model) directly to phytoplankton $(ip_ac_int[ip])")
                z_pos[:, iz] = p_pos[:, ip]

            elseif dist_3d[ip,iz] ≤ pred_visual_range #Predator cannot catch preys, but it sees them. Moves towards closest prey
                vec .= p_pos[:, ip] .- z_pos[:, iz]
                    #if dist_periodic[idim, ip, iz]
                    #    if vec[idim] > 0.0
                    #        vec[idim] = vec[idim] - 1.0
                    #    else
                    #        vec[idim] = vec[idim] + 1.0
                    #    end
                    #end
                #vec /= sqrt(sum(vec.^2))
                normalized_movement_vector = vec / dist_3d[ip,iz]
                
                z_pos[:, iz] = z_pos[:,iz] .+ normalized_movement_vector .* max_movement_len

            else #Predator moves in random vector
                for idim in 1:3
                    vec[idim] = 2* rand() -1 #Animal moves to random vector between -1 and 1

                    #if dist_periodic[idim, ip, iz] #Remove boundary effects?
                    #    if vec[idim] > 0.0
                    #        vec[idim] = vec[idim] - 1.0
                    #    else
                    #        vec[idim] = vec[idim] + 1.0
                    #    end
                    #end
                end
                #vec /= sqrt(sum(vec.^2))

                z_pos[:, iz] = z_pos[:,iz] .+ vec .* max_movement_len
            end

            model.individuals.phytos.sp2.data.x[iz_model] = z_pos[1, iz]
            model.individuals.phytos.sp2.data.y[iz_model] = z_pos[2, iz]
            model.individuals.phytos.sp2.data.z[iz_model] = z_pos[3, iz]
        end
    else
        ind_z = z_pause .> 0
        z_pause[ind_z] = max.(0.0, z_pause[ind_z] .- ΔT)
    end

    #
    # encounters and create plot
    #
    for i in 1:size(dist_3d,1)
        for j in 1:size(dist_3d,2)
            dist_3d[i,j] = sqrt((p_pos[1,i]-z_pos[1,j])^2+(p_pos[2,i]-z_pos[2,j])^2+(p_pos[3,i]-z_pos[3,j])^2)
        end
    end

    plt = Plots.scatter(p_pos[1, :], p_pos[2, :], p_pos[3,:], ms=4, color="#228833", markerstrokewidth=0, size=(900, 900), legend=:none)
    Plots.scatter!(z_pos[1, :], z_pos[2, :], z_pos[3,:], ms=5, color="#CCBB44", markerstrokewidth=0, legend=:none)

    encounters = findall(dist_3d .< encounter_tresh_sq)

    iz_vec = [ci.I[2] for ci in encounters]
    Plots.scatter!(z_pos[1, iz_vec], z_pos[2, iz_vec], z_pos[3,iz_vec], ms=5, color = "red", legend=:none)

    num_encounters = 0
    for ci in encounters
        # keep paused individuals unable to encounter
        if z_pause[iz_ac_int[ci.I[2]]] > 0.0
            continue
        end
        # ci is of type CartesianIndex
        Plots.scatter!([p_pos[1, ci.I[1]], z_pos[1, ci.I[2]]], [p_pos[2, ci.I[1]], z_pos[2, ci.I[2]]],[p_pos[3, ci.I[1]], z_pos[3, ci.I[2]]], ms=5, color="#EE6677", markerstrokewidth=0, legend=:none)
        Plots.scatter!([z_pos[1, ci.I[2]]], [z_pos[2, ci.I[2]]], [z_pos[3, ci.I[2]]], ms=5, color="#EE6677", markerstrokewidth=0, legend=:none)
        if deactivate
            #println("deactivating phytoplankton $(ip_ac_int[ci.I[1]])")
            model.individuals.phytos.sp1.data.ac[ip_ac_int[ci.I[1]]] = 0.0
        else
            model.individuals.phytos.sp1.data.x[ip_ac_int[ci.I[1]]] = rand()
            model.individuals.phytos.sp1.data.y[ip_ac_int[ci.I[1]]] = rand()
            model.individuals.phytos.sp1.data.z[ip_ac_int[ci.I[1]]] = rand()
        end
        z_pause[iz_ac_int[ci.I[2]]] = z_pause_ΔT
        num_encounters += 1
    end
    push!(encounter_diags["num_encounters"], num_encounters)

    Plots.xlims!(0, 1)
    Plots.ylims!(0, 1)
    Plots.zlims!(0, 1)
	return densities
end

## Run Test Function
This function assigns the proper diagnostic and state variables for the model to execute.

### Function Goals
1) Create the model grid. Note, the grid size is not actually used for movement calculations.
#
    grid = RectilinearGrid(size=(1,1,1),
                           x = (0, 10meters),
                           y = (0, 10meters),
                           z = (0,-10meters),
                           topology = (Periodic, Periodic, Periodic))
#
2) Assign model state parameters. This model uses the CarbonMode() from PlanktonIndividuals.jl and requires bgc parameters to operate. These variables do not actually do anything in this model.
#
    bgc_params = bgc_params_default()
    bgc_params["κhP"] = 1e-6
    bgc_params["κvP"] = 1e-6
    N_species = 2
    mode = CarbonMode()
    deactivate_relocate = "deactivate"
    if ! deactivate
        deactivate_relocate = "relocate"
    end
    phyt_params = phyt_params_default(N_species, mode)
    phyt_params["dvid_P"] = [0.0, 0.0]
    phyt_params["mort_P"] = [0.0, 0.0]
    ΔT = 60.0
    replicate_diags = Dict{String, Any}()
    replicate_diags["sum_encounters"] = Vector{Int64}()
    replicate_diags["mean_encounters"] = Vector{Float64}()
    z_pause = zeros(Float64, N_individual[2])
    preys = DataFrame(TS = [], Max = [], Min = [], Mean = [], SD = [])
#
3) Run each replicate for the simulation. The default model uses the PlanktonSimulation object and update!() functions from PlanktonIndividuals.jl. When calling the plot_model function, the prey densites are returned, but this can be adjusted. Currently, the plotting capabilities are commented out, as this increases model run time.
#
    for i = 1:N_replicates
        model = PlanktonModel(arch, grid; N_species = N_species,
                                          N_individual = N_individual,
                                          max_individuals = sum(N_individual),
                                          mode = mode,
                                          bgc_params = bgc_params,
                                          phyt_params = phyt_params)
        z_pause[1:end] .= 0.0
        pow = PlanktonOutputWriter(save_diags=true,
                                   save_plankton=true)
        sim = PlanktonSimulation(model, ΔT=ΔT, iterations = 1, output_writer=pow)
        encounter_diags = Dict{String, Any}()
        encounter_diags["num_encounters"] = Vector{Int64}()
        #anim = @animate for i in 1:N_iter
        for i in 1:N_iter
            update!(sim)
            densities = plot_model(model, z_pause, ΔT, deactivate, encounter_diags,densities,isubexp,i; z_pause_ΔT=z_pause_ΔT, encounter_tresh=encounter_tresh, velo1=velo1, velo2 = velo2,prey_vis=prey_vis,pred_vis = pred_vis)
            #plt = plot_model(model, z_pause, ΔT, deactivate, encounter_diags; z_pause_ΔT=z_pause_ΔT, encounter_tresh=encounter_tresh, velo1=velo1, velo2 = velo2,prey_vis=prey_vis,pred_vis = pred_vis)
            #if i < 10
            #    filename = "frames/frame_movement_$(deactivate_relocate)_3d_00$(i).png"
            #    Plots.savefig(plt, filename)
            end
            #plt 
        end
        #gif(anim, joinpath("frames", "exp $(isubexp).gif"), fps = 5) # used to be 15 fps
        push!(replicate_diags["sum_encounters"], sum(encounter_diags["num_encounters"]))
        push!(replicate_diags["mean_encounters"], mean(encounter_diags["num_encounters"]))
    end
#
4) Print the simulation characteristics and encounters to the REPL
#
    println("Prey swimming speed: $(velo2), n_prey = $(N_individual[1]), prey vis: $(prey_vis), ")
    println("number of encounters: $(replicate_diags["sum_encounters"]), mean number of encounters: $(replicate_diags["mean_encounters"])")
#

### Necessary Variables
- N_replicates: The number of iterations to run in parallel.
- N_individual: The number of individuals [preys,predator] that will exist at the start of the simulation.
- densities: The densities matrix from the previous function that is passed through this function
- z_pause_ΔT: The handling time for each predator, same as the previous function.
- velo1: Predator swimming velocity, same as the previous function.
- velo2: Prey swimming velocity, same as the previous function.
- encounter_tresh: The distance that a predator and prey need to be from each other foran encounter, same as previous function.
- deactivate: same as previous function
- arch: The architecture that is used. Generally this will be CPU()
- prey_vis: The proportion of the model volume the prey can perceive.
- pred_vis: The proportion of the model volume the predator can perceive.
- isubexp: A counter for the simulation number to set up result outputs
- N_iter: The total number of iterations each simulation will run.

In [ ]:
function run_test(N_replicates=10, N_individual=[2^8, 2^3]::Vector{Int64},densities = zeros()::Matrix{Int}; z_pause_ΔT=0.0::Float64, velo1=1e-4::Float64,velo2=1e-5::Float64, encounter_tresh=0.001::Float64, deactivate=false::Bool, arch=CPU(),prey_vis=1.0::Float64,pred_vis = 1.0::Float64,isubexp = 1::Int64,N_iter = 300::Int64)

    ## Can play around with grid. Scale grid size to larger distance to match predator velocity?
    grid = RectilinearGrid(size=(1,1,1),
                           x = (0, 10meters),
                           y = (0, 10meters),
                           z = (0,-10meters),
                           topology = (Periodic, Periodic, Periodic))


    bgc_params = bgc_params_default()
    bgc_params["κhP"] = 1e-6
    bgc_params["κvP"] = 1e-6

    N_species = 2
    mode = CarbonMode()

    deactivate_relocate = "deactivate"
    if ! deactivate
        deactivate_relocate = "relocate"
    end

    # set division and mortality to 0
    phyt_params = phyt_params_default(N_species, mode)
    phyt_params["dvid_P"] = [0.0, 0.0]
    phyt_params["mort_P"] = [0.0, 0.0]

    ΔT = 60.0

    replicate_diags = Dict{String, Any}()
    replicate_diags["sum_encounters"] = Vector{Int64}()
    replicate_diags["mean_encounters"] = Vector{Float64}()

    z_pause = zeros(Float64, N_individual[2])

    preys = DataFrame(TS = [], Max = [], Min = [], Mean = [], SD = [])

    for i = 1:N_replicates
        model = PlanktonModel(arch, grid; N_species = N_species,
                                          N_individual = N_individual,
                                          max_individuals = sum(N_individual),
                                          mode = mode,
                                          bgc_params = bgc_params,
                                          phyt_params = phyt_params)

        #model.individuals.phytos.sp1.data.ac[N_individual[1]+1:end] .= 0.0
        #model.individuals.phytos.sp2.data.ac[N_individual[1]+1:end] .= 0.0

        z_pause[1:end] .= 0.0

        pow = PlanktonOutputWriter(save_diags=true,
                                   save_plankton=true)

        sim = PlanktonSimulation(model, ΔT=ΔT, iterations = 1, output_writer=pow)
        encounter_diags = Dict{String, Any}()
        encounter_diags["num_encounters"] = Vector{Int64}()

        #anim = @animate for i in 1:N_iter
        for i in 1:N_iter
            update!(sim)

            densities = plot_model(model, z_pause, ΔT, deactivate, encounter_diags,densities,isubexp,i; z_pause_ΔT=z_pause_ΔT, encounter_tresh=encounter_tresh, velo1=velo1, velo2 = velo2,prey_vis=prey_vis,pred_vis = pred_vis)

            #plt = plot_model(model, z_pause, ΔT, deactivate, encounter_diags; z_pause_ΔT=z_pause_ΔT, encounter_tresh=encounter_tresh, velo1=velo1, velo2 = velo2,prey_vis=prey_vis,pred_vis = pred_vis)
            #if i < 10
            #    filename = "frames/frame_movement_$(deactivate_relocate)_3d_00$(i).png"
            #    Plots.savefig(plt, filename)
            #end
            #plt
            
        end
        #gif(anim, joinpath("frames", "exp $(isubexp).gif"), fps = 5) # used to be 15 fps

        #println(encounter_diags)
        push!(replicate_diags["sum_encounters"], sum(encounter_diags["num_encounters"]))
        push!(replicate_diags["mean_encounters"], mean(encounter_diags["num_encounters"]))
    end
    println("Prey swimming speed: $(velo2), n_prey = $(N_individual[1]), prey vis: $(prey_vis), ")
    println("number of encounters: $(replicate_diags["sum_encounters"]), mean number of encounters: $(replicate_diags["mean_encounters"])")

    return replicate_diags["sum_encounters"], N_iter, densities
end

## Run Experiments Function
This function is where the user adjusts model input parameters and the model runs each simulation in for loops.

### Function Goals
1) User defines the model inputs and other dataframes are created for naming and result saving purposes.
#
    res2 = DataFrame(Pred_velo = [], Prey_velo = [], PredN = [], PreyN = [], Speed = [], Prey_vis = [], Pred_vis = [], Encounter = [],Enc_sd = [])
    new_preys = []
    isubexp = 1
    ΔT = 60.0
    n_rep = 10
    encounter_tresh = 0.01
    n_z = 100 #Number of predators
    n_pz = [25:25:75;100:100:900;]
    n_it = 1
    velos1 = 1e-4 #Predator velocity
    pred_visual = 0.5
    velos2 = (1e-6,1e-5,3e-5,5e-5,7e-5,1e-4,3e-4,1e-3)
    prey_visual = [0:0.0001:0.0001;0.001;0.025:0.025:0.1;0.2:0.2:1;]
    z_pause_ΔT = 8.0*ΔT
    res = Dict{String, Dict}()
    expname = "exp_$(isubexp)"
    N_iter = 300
    densities = zeros(Int,n_z,length(n_pz)*length(velos2)*length(prey_visual))
#
2) Model runs all simulations one at a time. Two .csv files are output into the working directory, displaying the encounters and the prey densities. Each simulation is timed at this time is ouptut into the REPL so the user can predict the total run time when running multiple simulations. Note: The run time is related to the number of individuals in the model.
#
    for velo1 in velos1
        for pred_vis in pred_visual
                for velo2 in velos2
                    for prey_vis in prey_visual
                            for (i, n) in enumerate(n_pz)
                                start = now()
                                n_p = n
                                tmp, n_it, densities = run_test(n_rep, [n_p,n_z],densities; velo1=velo1, velo2 = velo2, z_pause_ΔT=z_pause_ΔT, encounter_tresh=encounter_tresh,prey_vis = prey_vis,pred_vis = pred_vis,isubexp,N_iter)
                                new_row = Dict("Pred_velo" => velo1, "Prey_velo" => velo2, "PredN" => n_z, "PreyN" => n_pz[i], "Speed" => velo1/velo2, "Prey_vis" => prey_vis, "Pred_vis" => pred_vis, "Encounter" => mean(tmp),"Enc_sd" => std(tmp))
                                push!(res2,new_row)
                                CSV.write("Prey visual range results.csv",res2)
                                CSV.write("Prey Densities.csv",Tables.table(densities), writeheader=false)
                        isubexp += 1
                        stop = now()
                        println(stop-start)
                    end
                end
            end
        end
    end
#

### Necessary Parameters
- No inputs required

In [ ]:
function run_experiments()
    res2 = DataFrame(Pred_velo = [], Prey_velo = [], PredN = [], PreyN = [], Speed = [], Prey_vis = [], Pred_vis = [], Encounter = [],Enc_sd = [])
    new_preys = []

    isubexp = 1

    ΔT = 60.0 # NOTE: not passed on, just used in computation for pause

    n_rep = 10 #Number of replicates to run
    encounter_tresh = 0.01  # increased encounter_tresh
    n_z = 100 #Number of predators
    # exponents = 6:12
    n_pz = [25:25:75;100:100:900;]

    n_it = 1
    #velos1 = (0, 3e-6, 1e-5, 3e-5, 1e-4) #Will want this to be data-derived and scaled to model area
    velos1 = 1e-4 #Predator velocity
    pred_visual = 0.5
    velos2 = (1e-6,1e-5,3e-5,5e-5,7e-5,1e-4,3e-4,1e-3) #Order of magnitude less swimming speed. Will want this to be data-derived

    prey_visual = [0:0.0001:0.0001;0.001;0.025:0.025:0.1;0.2:0.2:1;] #Prey visual ranges to test

    z_pause_ΔT = 8.0*ΔT
    res = Dict{String, Dict}()

    expname = "exp_$(isubexp)"
    N_iter = 300
    densities = zeros(Int,n_z,length(n_pz)*length(velos2)*length(prey_visual))

    for velo1 in velos1
        for pred_vis in pred_visual
                for velo2 in velos2
                    for prey_vis in prey_visual
                    #=
                            res[expname] = Dict(
                                "description" => "varying num(Prey) (Pred speed: $(velo1)) (Prey speed: $(velo2))",
                                "num_encounters" => fill(-1, (length(n_pz), n_rep)),
                                "num_prey" => Vector{Int64}(),
                                "num_pred" => n_z,
                                "pred_speed" => velo1,
                                "prey_speed" => velo2,
                                "encounter_tresh" => encounter_tresh,
                                "encounter_type" => "relocate",
                                "z_pause_ΔT" => z_pause_ΔT,
                            )
                            =#

                            for (i, n) in enumerate(n_pz)
                                start = now()

                                n_p = n
                                #push!(res[expname]["num_prey"], n_p)                        
                                tmp, n_it, densities = run_test(n_rep, [n_p,n_z],densities; velo1=velo1, velo2 = velo2, z_pause_ΔT=z_pause_ΔT, encounter_tresh=encounter_tresh,prey_vis = prey_vis,pred_vis = pred_vis,isubexp,N_iter)
                                #res[expname]["num_encounters"][i, :] = tmp
                                #res[expname]["num_iter"] = n_it

                                new_row = Dict("Pred_velo" => velo1, "Prey_velo" => velo2, "PredN" => n_z, "PreyN" => n_pz[i], "Speed" => velo1/velo2, "Prey_vis" => prey_vis, "Pred_vis" => pred_vis, "Encounter" => mean(tmp),"Enc_sd" => std(tmp))
                                push!(res2,new_row)
                                CSV.write("Prey visual range results.csv",res2)
                                CSV.write("Prey Densities.csv",Tables.table(densities), writeheader=false)
                        isubexp += 1
                        stop = now()
                        println(stop-start)
                    end
                end
            end
        end
    end
end

## Run the model

In [ ]:
run_experiments()